In [11]:
import pandas as pd
import json

def set_indx(df,form_id):
    temp_melted = df.melt(id_vars=['Store_id','year','month'],
                        var_name='que',           
                        value_name='ans')        

    temp_melted = temp_melted.dropna(subset=['ans'])
    temp_melted['count'] = 1
    temp_melted['form_id'] = form_id
    temp_melted = temp_melted.sort_values(['year','month'])
    temp_melted = temp_melted.groupby(
        ['form_id', 'Store_id', 'year', 'month', 'que', 'ans']
    ).agg({'count': 'sum'}).unstack([2,3,4, 5]).reset_index()
    return temp_melted

def question(forms,forms_sub,form_id,no_unique_values):
    que = forms[forms['id'] == form_id]['properties'].values[0]
    que = pd.read_json(que)
    t = forms_sub[forms_sub['form_id'] == 1]
    t = forms_sub[forms_sub['form_id'] == 1]
    res = t[['data','submission_user_id','store_id','created_at']]
    res['created_at'] = pd.to_datetime(res['created_at'], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')
    res['year'] = res['created_at'].dt.year
    res['month'] = res['created_at'].dt.month


    col = []
    first_row = json.loads(res['data'].values[0]) 
    col.extend(first_row.keys())  
    col.append('submission_user_id')
    col.append('Store_id')
    col.append('year')
    col.append('month')
    response = pd.DataFrame(columns=col)
    for a,id,stid,yr,mon in zip(res['data'].values, res['submission_user_id'].values, res['store_id'].values,res['year'].values,res['month'].values):
        data = json.loads(a)
        x = pd.json_normalize(data)
        x['submission_user_id'] = id
        x['Store_id'] = stid
        x['year'] = yr
        x['month'] = mon
        response = pd.concat([response, x], ignore_index=True, sort=False)
    
    col = ['submission_user_id','Store_id','year','month']
    for i in que['id'].values:
        col.append(i)

    Final_df = pd.DataFrame(columns=col)

    common_columns = Final_df.columns.intersection(response.columns)
    response_common = response[common_columns]
    Final_df = pd.concat([Final_df, response_common], ignore_index=True, sort=False)

    col = []
    for i in Final_df.columns:
        temp = Final_df[i].dropna()
        temp = temp.apply(lambda x: tuple(x) if isinstance(x, list) else x)
        uni = temp.unique() 
        if len(uni) < no_unique_values:
            col.append(i)
    

    col.remove('year')
    col.remove('month')

    x = col.copy()
    x.append('submission_user_id')
    x.append('Store_id')
    x.append('year')
    x.append('month')

    temp = Final_df[x]
    up_col = []
    for i in col:
        up_col.append(que[que['id'] == i]['name'].values[0])
    up_col.append('submission_user_id')
    up_col.append('Store_id')
    up_col.append('year')
    up_col.append('month')
    temp.columns = up_col
    temp = temp.drop(columns=['submission_user_id'])
    return set_indx(temp,form_id)




In [12]:
forms = pd.read_csv('forms_202501301141.csv')
forms_sub = pd.read_csv('form_submissions_202501301145.csv')

# Morning Store Checklist

In [13]:
form_67 = question(forms,forms_sub,form_id = 67,no_unique_values = 10)
form_67

C:\Users\Medkart\AppData\Local\Temp\ipykernel_4228\765197439.py:20: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  que = pd.read_json(que)
C:\Users\Medkart\AppData\Local\Temp\ipykernel_4228\765197439.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['created_at'] = pd.to_datetime(res['created_at'], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')
C:\Users\Medkart\AppData\Local\Temp\ipykernel_4228\765197439.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

form_id Store_id           count                              \
year                              2024                               
month                                7                               
que                    Are Bins clean? Are all IT Systems Working?   
ans                                Yes                         Yes   
0          67      1.0             5.0                         5.0   
1          67      2.0             2.0                         1.0   
2          67      3.0             5.0                         5.0   
3          67      5.0             2.0                         2.0   
4          67      6.0             5.0                         5.0   
..        ...      ...             ...                         ...   
63         67    124.0             3.0                         3.0   
64         67    125.0             2.0                         2.0   
65         67    126.0             2.0                         2.0   
66         67    127.0            25.0                        25.0   
67         67    130.0             NaN                         NaN   

                                                                              \
year                                                                           
month                                                                          
que   Are spider webs visible? Are tea marks visible? Area Outside The Store   
ans                         No                     No       Clean & Hygienic   
0                          5.0                    5.0                    4.0   
1                          2.0                    2.0                    2.0   
2                          5.0                    5.0                    5.0   
3                          2.0                    2.0                    2.0   
4                          3.0                    5.0                    5.0   
..                         ...                    ...                    ...   
63                         3.0                    3.0                    3.0   
64                         2.0                    2.0                    2.0   
65                         2.0                    2.0                    2.0   
66                        24.0                   25.0                   24.0   
67                         NaN                    NaN                    NaN   

                                                                ...  \
year                                                            ...   
month                                                           ...   
que             Arrangement of Bins Counter and Counter Shield  ...   
ans   Not Clean        Well Aligned           Clean & Hygienic  ...   
0           1.0                 5.0                        5.0  ...   
1           NaN                 2.0                        2.0  ...   
2           NaN                 5.0                        5.0  ...   
3           NaN                 2.0                        2.0  ...   
4           NaN                 5.0                        5.0  ...   
..          ...                 ...                        ...  ...   
63          NaN                 3.0                        3.0  ...   
64          NaN                 2.0                        2.0  ...   
65          NaN                 2.0                        2.0  ...   
66          1.0                25.0                       25.0  ...   
67          NaN                 NaN                        NaN  ...   

                                                                          \
year                                                                       
month                  12                                             11   
que   Arrangement of Bins Racks and shelfs are clean Arrangement of Bins   
ans            Not Proper                         No          Not Proper   
0                     NaN                        NaN                 NaN   
1          

# Evening Store Checklist

In [14]:
form_1 = question(forms,forms_sub,form_id = 1,no_unique_values = 10)
form_1

C:\Users\Medkart\AppData\Local\Temp\ipykernel_4228\765197439.py:20: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  que = pd.read_json(que)
C:\Users\Medkart\AppData\Local\Temp\ipykernel_4228\765197439.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['created_at'] = pd.to_datetime(res['created_at'], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')
C:\Users\Medkart\AppData\Local\Temp\ipykernel_4228\765197439.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

form_id Store_id           count                              \
year                              2024                               
month                                7                               
que                    Are Bins clean? Are all IT Systems Working?   
ans                                Yes                         Yes   
0           1      1.0             5.0                         5.0   
1           1      2.0             2.0                         1.0   
2           1      3.0             5.0                         5.0   
3           1      5.0             2.0                         2.0   
4           1      6.0             5.0                         5.0   
..        ...      ...             ...                         ...   
63          1    124.0             3.0                         3.0   
64          1    125.0             2.0                         2.0   
65          1    126.0             2.0                         2.0   
66          1    127.0            25.0                        25.0   
67          1    130.0             NaN                         NaN   

                                                       \
year                                                    
month                                                   
que   Are spider webs visible? Are tea marks visible?   
ans                         No                     No   
0                          5.0                    5.0   
1                          2.0                    2.0   
2                          5.0                    5.0   
3                          2.0                    2.0   
4                          3.0                    5.0   
..                         ...                    ...   
63                         3.0                    3.0   
64                         2.0                    2.0   
65                         2.0                    2.0   
66                        24.0                   25.0   
67                         NaN                    NaN   

                                                                   \
year                                                                
month                                                               
que   Area Outside The Store(Daily)           Arrangement of Bins   
ans                Clean & Hygienic Not Clean        Well Aligned   
0                               4.0       1.0                 5.0   
1                               2.0       NaN                 2.0   
2                               5.0       NaN                 5.0   
3                               2.0       NaN                 2.0   
4                               5.0       NaN                 5.0   
..                              ...       ...                 ...   
63                              3.0       NaN                 3.0   
64                              2.0       NaN                 2.0   
65                              2.0       NaN                 2.0   
66                             24.0       1.0                25.0   
67                              NaN       NaN                 NaN   

                                  ...  \
year                              ...   
month                             ...   
que   Counter and Counter Shield  ...   
ans             Clean & Hygienic  ...   
0                            5.0  ...   
1                            2.0  ...   
2                            5.0  ...   
3                            2.0  ...   
4                            5.0  ...   
..                           ...  ...   
63                           3.0  ...   
64                           2.0  ...   
65                           2.0  ...   
66                          25.0  ...   
67                           NaN  ...   

                                                \
year                                             
month                                       10   
que   Which IT system is not working properly?   
ans       

# ASM Checklist

In [5]:
# form_16 = question(forms,forms_sub,form_id = 16,no_unique_values = 10)
# form_16